# Tokenization

In [80]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

--2024-09-23 00:53:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.30MB/s    in 0.5s    

2024-09-23 00:53:46 (2.30 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [81]:
tokens = list(map(int, text.encode('utf-8')))
print(len(tokens))

1115394


In [82]:
from collections import Counter

def merge(text, pair, idx):
    new_text = [text[0]]
    for i in range(1, len(text)):
        if (text[i - 1], text[i]) == pair:
            new_text[-1] = idx
        else:
            new_text.append(text[i])
    return new_text

VOCAB_SIZE = 1000

idx = 256
merges = {}
while idx < VOCAB_SIZE:
    pair = Counter(zip(tokens, tokens[1:])).most_common(1)[0][0]
    tokens = merge(tokens, pair, idx)
    merges[pair] = idx
    print(f"{pair} -> {idx}")
    idx += 1

(101, 32) -> 256
(116, 104) -> 257
(116, 32) -> 258
(115, 32) -> 259
(100, 32) -> 260
(44, 32) -> 261
(111, 117) -> 262
(101, 114) -> 263
(105, 110) -> 264
(121, 32) -> 265
(97, 110) -> 266
(58, 10) -> 267
(111, 114) -> 268
(111, 32) -> 269
(101, 110) -> 270
(10, 10) -> 271
(97, 114) -> 272
(32, 257) -> 273
(111, 110) -> 274
(108, 108) -> 275
(104, 97) -> 276
(44, 10) -> 277
(46, 271) -> 278
(105, 259) -> 279
(101, 115) -> 280
(121, 262) -> 281
(32, 115) -> 282
(116, 269) -> 283
(266, 260) -> 284
(111, 119) -> 285
(101, 97) -> 286
(32, 109) -> 287
(32, 119) -> 288
(111, 102) -> 289
(32, 104) -> 290
(264, 103) -> 291
(111, 109) -> 292
(32, 97) -> 293
(99, 104) -> 294
(257, 256) -> 295
(115, 116) -> 296
(32, 98) -> 297
(110, 111) -> 298
(105, 114) -> 299
(102, 268) -> 300
(118, 256) -> 301
(101, 261) -> 302
(105, 257) -> 303
(273, 256) -> 304
(115, 101) -> 305
(108, 105) -> 306
(84, 104) -> 307
(275, 32) -> 308
(114, 101) -> 309
(115, 258) -> 310
(97, 258) -> 311
(65, 110) -> 312
(73, 32

In [83]:
print(len(tokens))

447065


In [84]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for pair, idx in merges.items():
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]



def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text
        
decode([299])

'ir'

In [97]:
def encode(string):
    tokens = list(map(int, string.encode('utf-8')))
    while True:
        new_tokens = []
        i = 0
        merges_count = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i + 1]) in merges:
                new_tokens.append(
                    merges[tokens[i], tokens[i + 1]]
                )
                i += 2
                merges_count += 1
            else:
                new_tokens.append(tokens[i])
                i += 1
        if merges_count == 0:
            return new_tokens
        tokens = new_tokens

decode(encode("hello world!"))

'hello world!'